<a href="https://colab.research.google.com/github/jingxiangmo/ml-algorithms/blob/main/lunarlanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt install swig cmake
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt
!sudo apt-get update
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (4.0.1-5build1).
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers

In [1]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [2]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import (
    notebook_login,
)

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

In [6]:
import gymnasium as gym

# create LUnarLander-v2 environment
env = gym.make("LunarLander-v2")

# reset environment
observation, info = env.reset()

for _ in range(20):
  # take random action
  action = env.action_space.sample()
  print("Action: ", action)

  # action in environment => next_state, reward
  observation, reward, terminated, truncated, info = env.step(action)

  # if game is terminated
  if terminated or truncated:
    # reset environment
    print("environment reset")
    observation, info = env.reset()
  
  env.close()

Action:  2
Action:  2
Action:  0
Action:  0
Action:  2
Action:  3
Action:  3
Action:  2
Action:  3
Action:  1
Action:  0
Action:  2
Action:  0
Action:  2
Action:  0
Action:  0
Action:  2
Action:  1
Action:  2
Action:  3


In [8]:
env = gym.make("LunarLander-v2")
env.reset()

print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())  # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())  # Take a random action

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [31.487411   34.92894    -2.7800524   1.4671094  -0.07549364 -3.7656708
  0.24626766  0.18907456]

 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 0


In [9]:
# create vertorized environment
env = make_vec_env("LunarLander-v2", n_envs=16)

In [12]:
# Stable-Baseline3

# create environment
env = gym.make('LunarLander-v2')

# instantiate the agent with PPO MlpPolicy architecture
model = PPO( 
    policy="MlpPolicy",
    env=env,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1,
)

# train agent
model.learn(total_timesteps=1000)

# save model
model_name = "ppo-LunarLander-v2"
model.save(model_name)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 85.5     |
|    ep_rew_mean     | -176     |
| time/              |          |
|    fps             | 725      |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 1024     |
---------------------------------


In [14]:
# evaluate agent
eval_env = gym.make("LunarLander-v2")
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=-372.46 +/- 112.0950263976835
